## Notebook parameters

In [28]:
# Add the parameters tag to this cell
# kedro_env = 'local'
kedro_env = 'test_cloud'

In [29]:
relative_project_path = '..//'

## Imports

In [23]:
import yaml
import pandas as pd
from kedro_datasets.pandas import GBQTableDataSet, CSVDataSet
from kedro.io import PartitionedDataSet
from typing import Dict

## Loads kedro yml data

In [24]:
from kedro.config import ConfigLoader
from kedro.framework.project import settings

KEDRO_ENV = "local"
conf_path = str("../" + settings.CONF_SOURCE)
conf_loader = ConfigLoader(conf_source=conf_path, env=kedro_env)

conf_catalog = conf_loader["catalog"]

[05/18/23 12:25:26] INFO     Config from path '../conf/test_cloud' will override the following         ]8;id=846121;file:///home/anfef/Codes/2023/Species/species-observations/.venv/lib/python3.10/site-packages/kedro/config/common.py\common.py]8;;\:]8;id=587290;file:///home/anfef/Codes/2023/Species/species-observations/.venv/lib/python3.10/site-packages/kedro/config/common.py#93\93]8;;\
                             existing top-level config keys: species_data                                          

## Query using bigquery API for query debugging

In [25]:
# bq_query_cat_entry = catalog['bq_test']
# project_id = bq_query_cat_entry['project']

# Recovers contents of ARRAYS nested inside STRUCTS
# sql = """SELECT element.array_element
#     FROM bigquery-public-data.gbif.occurrences,
#     UNNEST(issue.array) AS element
#     LIMIT 20"""

# Shows the distinct values of the issue column, which is an ARRAY nested inside a STRUCT
# sql = """SELECT distinct element.array_element
#     FROM bigquery-public-data.gbif.occurrences,
#     UNNEST(issue.array) AS element"""

# Counts the number of observations per species and orders it from high to low
# sql = """ SELECT species,
#   count(species) as cnt
#   FROM bigquery-public-data.gbif.occurrences
#   group by species
#   order by cnt desc"""

# Counts the number of observations per species and orders it from high to low
# sql = """ SELECT species,
#   count(species) as cnt
#   FROM bigquery-public-data.gbif.occurrences
#   group by species
#   order by cnt desc"""

# Gets top 20 rows of all data
# sql = """ SELECT gbifid
#     ,datasetkey
#     ,occurrenceid
#     ,kingdom
#     ,phylum
#     ,class
#     ,`order`
#     ,family
#     ,genus
#     ,species
#     ,infraspecificepithet
#     ,taxonrank
#     ,scientificname
#     ,verbatimscientificname
#     ,verbatimscientificnameauthorship
#     ,countrycode
#     ,locality
#     ,stateprovince
#     ,occurrencestatus
#     ,individualcount
#     ,publishingorgkey
#     ,decimallatitude
#     ,decimallongitude
#     ,coordinateuncertaintyinmeters
#     ,coordinateprecision
#     ,elevation
#     ,elevationaccuracy
#     ,depth
#     ,depthaccuracy
#     ,eventdate
#     ,day
#     ,month
#     ,year
#     ,taxonkey
#     ,specieskey
#     ,basisofrecord
#     ,institutioncode
#     ,collectioncode
#     ,catalognumber
#     ,recordnumber
#     ,identifiedby
#     ,dateidentified
#     ,license
#     ,rightsholder
#     ,recordedby
#     ,typestatus
#     ,establishmentmeans
#     ,lastinterpreted
#     ,mediatype
#     ,issue
# FROM bigquery-public-data.gbif.occurrences
# LIMIT 20 """

# 
# sql = """SELECT gbifid
#     ,kingdom
#     ,phylum
#     ,class
#     ,`order`
#     ,family
#     ,genus
#     ,species
#     ,countrycode
#     ,occurrencestatus
#     ,individualcount
#     ,decimallatitude
#     ,decimallongitude
#     ,eventdate
#     ,day
#     ,month
#     ,year
#     ,basisofrecord 
#     ,element.array_element as issue
#     FROM bigquery-public-data.gbif.occurrences
#     CROSS JOIN UNNEST(issue.array) AS element
#     WHERE species = 'Anas platyrhynchos' AND eventdate > '2020-01-01'
#     """

# data_set = GBQQueryDataSet(sql, project=project_id)
# df = data_set.load()

## Loads csv with data downladed from bigquery

In [30]:
catalog_entry = 'species_data'
path = relative_project_path + conf_catalog[catalog_entry]['path']

In [31]:
def load_kedro_partitionedDS(path: str, dataset: Dict) -> pd.DataFrame:
    """Loads as a single dataframe the partitioned data stored in path.

    Parameters
    ----------
    path : str
        Path of the partitioned data
    dataset : Dict
        Type of data to search for and load options (dataset option in kedro's catalog of type PartitionedDataSet)

    Returns
    -------
    pd.DataFrame
        joined dataset
    """
    data_set = PartitionedDataSet(
        path=path,
        dataset=dataset,
    )

    print(dataset)

    loaded = data_set.load()

    df = pd.DataFrame()
    for partition_id, partition_load_func in loaded.items():
        partition_data = partition_load_func()
        df = pd.concat(
            [df, partition_data], ignore_index=True, sort=True
        )
    return df 



In [32]:
if kedro_env == 'local':
    dataset = conf_catalog[catalog_entry]['dataset']
    df = load_kedro_partitionedDS(path, dataset)
elif kedro_env == 'test_cloud':
    data_set = CSVDataSet(filepath=path)
    df = data_set.load()
else:
    raise ValueError('Undefined kedro environment')

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        1000 non-null   int64  
 1   basisofrecord     1000 non-null   object 
 2   class             1000 non-null   object 
 3   countrycode       1000 non-null   object 
 4   day               1000 non-null   float64
 5   decimallatitude   1000 non-null   float64
 6   decimallongitude  1000 non-null   float64
 7   eventdate         1000 non-null   object 
 8   family            1000 non-null   object 
 9   gbifid            1000 non-null   int64  
 10  genus             1000 non-null   object 
 11  individualcount   903 non-null    float64
 12  issue             1000 non-null   object 
 13  kingdom           1000 non-null   object 
 14  month             1000 non-null   float64
 15  occurrencestatus  1000 non-null   object 
 16  order             1000 non-null   object 
 

## Generates sample data for automatic testing

In [8]:
# NBVAL_SKIP
# test_catalog_entry = 'species_data'
# test_path = '..//' + conf_catalog[test_catalog_entry]['path']

# sample_index = combine_all.species.sample(1000).index
# sample_df = combine_all.loc[sample_index]
# sample_df.to_csv(test_path)

In [9]:
# sample_df